In [6]:
# import all the libraries used for the program

import os
import cv2
import cvlib as cv
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file

Using TensorFlow backend.


In [7]:
# reading image from the path
image = cv2.imread('full size images\\group.jpg')
# detecting faces using open-cv
faces, _ = cv.detect_face(image)

print("Number of faces detected in the image: ",len(faces))

Number of faces detected in the image:  8


In [8]:
# loading the classification model from the path

model_path = os.path.join(os.path.dirname(os.getcwd()),'Train Classifier\\3-conv-95.h5')
model = load_model(model_path)

In [9]:
# function that takes image and the coordinates of faces and model as input
# and returns the image with marked faces and label the gender of faces

def markFace(faces, image, model):
    classes = ['man', 'woman']
    if len(faces) == 0:
        return image
    
    for face in faces:
         # get corner points of face rectangle       
        (startX, startY) = face[0], face[1]
        (endX, endY) = face[2], face[3]

        # crop the detected face region
        face_crop = np.copy(image[startY:endY,startX:endX])

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (150,150))
        face_crop = face_crop / 255.0
        face_crop = np.reshape(face_crop,(1,150,150,3))

        # predicting the gender of the cropped face
        conf = model.predict(face_crop)[0][0]

        # get label of the image
        idx = int(np.round(conf))
        label = classes[idx]

        # get the amount of confidence in the prediction
        confid = conf if idx == 1 else 1-conf

        label = "{}: {:.2f}%".format(label, confid * 100)


        # if there is no space above rectangle then write below
        Y = startY - 10 if startY - 10 > 10 else endY + 10

        # draw rectangle over face
        cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)
        # write label and confidence above face rectangle
        fontScale = (endX-startX)/(35/0.15)
        if fontScale < 0.5: fontScale = 0.5
        cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 255, 0), 2)
        
    return image

In [10]:
# calling markFace function on the given image
image = markFace(faces, image, model)

# showing output image in terminal
cv2.imshow("output", image)
cv2.waitKey()

# save image
cv2.imwrite("output image\\group.jpg", image)

# release resources
cv2.destroyAllWindows()